In [2]:
pip install -U langchain langchain-community

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install langchain openai pypdf faiss-cpu tiktoken

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install pdfplumber

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [5]:
import pdfplumber
import openai
import os

# ✅ OpenAI API 키 (환경변수 또는 직접 입력)
openai.api_key = os.getenv("OPENAI_API_KEY")

# ✅ OpenAI 모델명
MODEL = "gpt-4o-mini"

# ✅ PDF에서 텍스트 추출 함수
def extract_text_from_pdf(pdf_path: str) -> str:
    all_text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            if text:
                all_text += text + "\n"
    return all_text

# ✅ GPT로 질문에 답변 요청 함수
def ask_question_about_pdf(text: str, question: str) -> str:
    messages = [
        {"role": "system", "content": "다음은 PDF 문서의 전체 내용입니다. 이 내용을 바탕으로 질문에 답하세요."},
        {"role": "user", "content": f"문서 내용:\n{text}\n\n질문: {question}"}
    ]
    response = openai.chat.completions.create(
        model=MODEL,
        messages=messages,
        temperature=0.5,
        max_tokens=1024
    )
    return response.choices[0].message.content

# ✅ 전체 실행 함수
def answer_from_pdf(pdf_path: str, user_question: str):
    print(f"📄 '{pdf_path}'에서 텍스트를 추출 중...")
    text = extract_text_from_pdf(pdf_path)
    if not text.strip():
        print("❌ 문서에서 텍스트를 추출할 수 없습니다.")
        return

    print("🤖 GPT에게 질문 중...")
    answer = ask_question_about_pdf(text, user_question)
    print("\n📌 GPT의 답변:\n")
    print(answer)

# ✅ 예시 실행
if __name__ == "__main__":
    answer_from_pdf("소나기.pdf", "이 문서의 핵심 내용은 무엇인가요?")

📄 '소나기.pdf'에서 텍스트를 추출 중...
🤖 GPT에게 질문 중...

📌 GPT의 답변:

이 문서의 핵심 내용은 소년과 소녀의 순수한 우정과 성장 과정을 담고 있습니다. 소년은 개울가에서 소녀를 만나 물장난을 하며 즐거운 시간을 보내고, 둘 사이의 관계가 점차 깊어지는 과정을 그립니다. 그러나 소녀가 이사 간다는 소식이 전해지면서 소년은 상실감과 슬픔을 느끼게 됩니다. 이 이야기는 어린 시절의 순수한 감정과 첫사랑의 아련함을 표현하고 있으며, 소년이 소녀와의 기억을 소중히 여기고 있다는 점이 강조됩니다.


In [6]:
import pdfplumber
import openai
import os
import gradio as gr

# ✅ OpenAI API 키 (환경변수에 저장되어 있어야 함)
openai.api_key = os.getenv("OPENAI_API_KEY")

# ✅ 모델 이름 고정
MODEL = "gpt-4o-mini"

# ✅ PDF 텍스트 추출 함수
def extract_text_from_pdf(pdf_path: str) -> str:
    all_text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            if text:
                all_text += text + "\n"
    return all_text

# ✅ GPT 응답 함수
def ask_question_about_pdf(text: str, question: str) -> str:
    messages = [
        {"role": "system", "content": "다음은 사용자가 업로드한 PDF 문서의 내용입니다. 이 내용을 기반으로 질문에 정확하고 간결하게 답해주세요."},
        {"role": "user", "content": f"문서 내용:\n{text}\n\n질문: {question}"}
    ]
    response = openai.chat.completions.create(
        model=MODEL,
        messages=messages,
        temperature=0.5,
        max_tokens=1024
    )
    return response.choices[0].message.content

# ✅ Gradio 인터페이스 처리 함수
def chat_with_pdf(pdf_file, question):
    if pdf_file is None:
        return "❌ PDF 파일을 업로드해주세요."
    if not question.strip():
        return "❌ 질문을 입력해주세요."

    try:
        text = extract_text_from_pdf(pdf_file.name)
        if not text.strip():
            return "❌ 문서에서 텍스트를 추출할 수 없습니다."
        answer = ask_question_about_pdf(text, question)
        return answer
    except Exception as e:
        return f"⚠️ 오류 발생: {str(e)}"

# ✅ Gradio UI 구성
with gr.Blocks(title="📄 PDF 기반 GPT 질의응답") as demo:
    gr.Markdown("## 📚 업로드한 PDF 문서에 기반한 ChatGPT 질의응답 (gpt-4o-mini 사용)")
    with gr.Row():
        pdf_input = gr.File(label="📎 PDF 업로드", file_types=[".pdf"])
        question_input = gr.Textbox(label="💬 질문 입력", placeholder="예: 이 문서의 핵심 내용은 무엇인가요?")
    output = gr.Textbox(label="🤖 GPT의 응답")

    submit_btn = gr.Button("질문하기")
    submit_btn.click(chat_with_pdf, inputs=[pdf_input, question_input], outputs=output)

# ✅ 앱 실행
if __name__ == "__main__":
    demo.launch()
    

/home/ubuntu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
